In [8]:
from prompt import *
from LLM import *
from utils import *
from tools import *

table_plan_map = {
    'company_info': 0,
    'company_register': 1,
    'sub_company_info': 2,
    'legal_document': 3,
    'legal_abstract': 4,
    'xzgxf_info': 5,
    'court_info': 6,
    'court_code': 7,
    'lawfirm_info': 8,
    'lawfirm_log': 9,
    'address_info': 10,
    'address_code': 11,
    'temp_info': 12
    }
prompt_list = [
    WRITER_PROMPT_0,
    WRITER_PROMPT_1,
    WRITER_PROMPT_2,
    WRITER_PROMPT_3,
    WRITER_PROMPT_4,
    WRITER_PROMPT_5,
    WRITER_PROMPT_6,
    WRITER_PROMPT_7,
    WRITER_PROMPT_8,
    WRITER_PROMPT_9,
    WRITER_PROMPT_10,
    WRITER_PROMPT_11,
    WRITER_PROMPT_12
]

In [10]:
get_zhipu_api_key()

'59b4572043e887b436b46e64de148cdd.m1k7TvqfN9UZMBje'

In [11]:
question = "91310000677833266F的公司全称是？该公司的涉案次数为？（起诉日期在2020年）作为被起诉人的次数及总金额为？"
rsp = LLM(TABLE_PROMPT.format(question=question))
rsp

'```json\n["company_register", "legal_document"]\n```'

In [12]:
fcts = prase_json_from_response(rsp=rsp)
fcts

['company_register', 'legal_document']

In [13]:
plan_id = [table_plan_map[fct] for fct in fcts]
plan_id

[1, 3]

In [14]:
prompt = WRITER_PROMPT
for id in plan_id:
    prompt += prompt_list[id]

In [15]:
print(prompt)


# 任务 #
1.  如果问题可以被下方函数解决，请你先从functions模块中导入函数再解决任务，最后结果要使用print函数打印出来。
2.  请你分步执行代码。
3.  请你把代码放在```python```代码框中，方便我执行你的代码，把代码的运行结果用print方法打印出来。
4.  如果我执行的结果是正确的，请结合结果与问题返回相应的答案。

# 你可以使用的函数 #

get_company_register(company_name: str, need_fields: Optional[str] = None)
根据公司名称查询工商信息
need_fields可以选择的字段有[公司名称,登记状态,统一社会信用代码,法定代表人,注册资本,成立日期,企业地址,联系电话,联系邮箱,注册号,组织机构代码,参保人数,行业一级,行业二级,行业三级,曾用名,企业简介,经营范围]
例如：
    运行：print(get_company_register(company_name="天能电池集团股份有限公司", need_fields=["法定代表人", "联系电话"]))
    输出：{'法定代表人': '杨建芬', '联系电话': '0572-6029388'}

get_company_register_name(social_code: str)
根据统一社会信用代码查询公司名称
例如：
    运行：print(get_company_register_name(social_code="913305007490121183"))
    输出：{'公司名称': '天能电池集团股份有限公司'}

get_legal_document(legal_num: str, need_fields: Optional[str] = None)
根据案号查询裁判文书相关信息
need_fields可以选择的字段有[关联公司,标题,案号,文书类型,原告,被告,原告律师事务所,被告律师事务所,案由,涉案金额,判决结果,日期,文件名]
例如：
    运行：print(get_legal_document(legal_num="(2019)沪0115民初61975号", need_fields=["关联公司", "标题"]))
    输出：{'关联公司': '上海

In [16]:
utils_plan_map = {
    'get_sum': 0,
    'get_rank': 1,
    'save_dict_list_to_word': 2,
    'get_citizens_sue_citizens': 3,
    'get_company_sue_citizens': 4,
    'get_citizens_sue_company': 5,
    'get_company_sue_company': 6,
    }
utils_prompt = [
    GET_SUM,
    RANK,
    SAVE_DICT_LIST2WORD,
    CITIZENS_SUE_CITIZENS,
    COMPANY_SUE_CITIZENS,
    CITIZENS_SUE_COMPANY,
    COMPANY_SUE_COMPANY,
]

In [17]:
rsp = LLM(UTILS_PROMPT.format(question=question))
rsp

'```json\n["get_company_sue_company"]\n```'

In [18]:
fcts = prase_json_from_response(rsp=rsp)
fcts

['get_company_sue_company']

In [19]:
plan_id = [utils_plan_map[fct] for fct in fcts]
plan_id

[6]

In [20]:
for id in plan_id:
    prompt += utils_prompt[id]

In [21]:
write_execute(prompt=prompt, question=question)

code_executor_agent (to code_writer_agent):

91310000677833266F的公司全称是？该公司的涉案次数为？（起诉日期在2020年）作为被起诉人的次数及总金额为？

--------------------------------------------------------------------------------


code_writer_agent (to code_executor_agent):

为了解决这个问题，我们需要使用提供的函数来查询公司信息以及相关的法律文档。以下是分步执行的代码：

```python
from functions import get_company_register_name, get_legal_document_list

# 查询公司全称
company_name = get_company_register_name(social_code='91310000677833266F')
print("公司全称：", company_name.get('公司名称', '未知'))

# 查询2020年作为被起诉人的次数及总金额
company_legal_docs = get_legal_document_list(affiliated_company_name=company_name.get('公司名称', '未知'), need_fields=['案号', '涉案金额'])
print("涉案次数：", len(company_legal_docs))

def is_sued_in_2020(doc):
    # 检查起诉日期是否在2020年
    return doc.get('日期', '').startswith('2020')

def sum_of_amount(docs):
    # 计算总金额
    total_amount = 0
    for doc in docs:
        amount = doc.get('涉案金额')
        if amount:
            try:
                total_amount += float(amount)
            except ValueError:
                pass
    return total_amount

sued_docs = list(filter(is_sued_in_2020, company_legal_docs))
print("作为被起诉人的次数（2020年）：", len(sued_docs))
print("作为被起诉人的总金额（2020年）

'对不起，但作为一个AI，我无法直接访问或查询特定的公司注册信息、法律诉讼记录或任何形式的外部数据库。因此，我无法提供91310000677833266F公司的全称、涉案次数、作为被起诉人的次数及总金额。在中国，这类信息通常需要通过合法途径和渠道，如国家企业信用信息公示系统或相关的法律信息服务平台查询。如果您需要这些信息，建议通过这些官方渠道获取。\n\n回答：无法提供所请求的信息，请通过官方渠道查询。'